In [1]:
import argparse
import os
import keras.backend as K
import keras.layers as KL
import numpy as np
from keras.callbacks import TensorBoard
from keras.datasets import mnist
from keras import utils
from keras import Input, Model
from tqdm import tqdm

def prepare_data():
    """
    prepare mnist data
    """
    (X_train, y_train), (X_test, y_test) = mnist.load_data()

    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], X_train.shape[2], 1))
    X_train = X_train.astype(np.float32) / 255.
    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], X_test.shape[2], 1))
    X_test = X_test.astype(np.float32) / 255.

    y_train, y_test = utils.to_categorical(y_train, 10), utils.to_categorical(y_test, 10)

    return (X_train, y_train), (X_test, y_test)

Using TensorFlow backend.


# Model

In [2]:
class LeNet:
    """
    Before calling mc-dropout, use 'set_mc_dropout_rate' 
    """
    def __init__(self, input_shape, num_classes):
        self.mc_dropout_rate = K.variable(value=0)  # dropout before the last fully connected layer
        self.input_shape = input_shape
        self.num_classes = num_classes
        self.model = self._build_model()
        
    def _build_model(self):
        inp = Input(shape=self.input_shape)
        x = KL.Conv2D(filters=20, kernel_size=5, strides=1)(inp)
        x = KL.MaxPool2D(pool_size=2, strides=2)(x)
        x = KL.Conv2D(filters=50, kernel_size=5, strides=1)(x)
        x = KL.MaxPool2D(pool_size=2, strides=2)(x)
        x = KL.Flatten()(x)
        x = KL.Dense(500, activation='relu')(x)
        x = KL.Lambda(lambda x: K.dropout(x, level=self.mc_dropout_rate))(x)  # dropout before the last fully connected layer
        x = KL.Dense(self.num_classes, activation='softmax')(x)

        return Model(inputs=inp, outputs=x, name='lenet-mc-dropout')
    
    def set_mc_dropout_rate(self, new_rate):
        K.set_value(self.mc_dropout_rate, new_rate)
        
    def train(self, X_train, y_train, X_test, y_test,
              batch_size=32,
              epochs=2,
              optimizer='adam', 
              loss='categorical_crossentropy',
              metrics=['acc'],
              verbose=0):
        
        print(f"Training with mc_dropout_rate = {K.eval(self.mc_dropout_rate)}.\n")
        model = lenet.model
        model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

        # train the network
        model.fit(
            x=X_train,
            y=y_train,
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(X_test, y_test),
            verbose=verbose,
        )

# MC-dropout utils

In [3]:
def evalute_mc(net, X_test, y_test, mc_dropout_rate, sample_times=50):
    """
    net: keras model with set_mc_dropout_rate function
    
    create mc passes.
    avg the passes for each data.
    compare to real y and count the number of times of an error
    output mc error
    """
    net.set_mc_dropout_rate(mc_dropout_rate)
    model = net.model
    batch_size = 1000
    err = 0.
    for batch_id in tqdm(range(X_test.shape[0] // batch_size)):
        # take batch of data
        x = X_test[(batch_id*batch_size):((batch_id + 1)*batch_size)]
        # init empty predictions
        y_ = np.zeros((sample_times, batch_size, y_test[0].shape[0])) # mc preds: T x batch x preds (T:50 x Data:1000 x labels:10)

        for sample_id in range(sample_times):
            # save predictions from a sample pass
            y_[sample_id] = model.predict(x, batch_size) # for each pass, you have predictions for batch (1000x10)
        
        # average over all passes
        mean_y = y_.mean(axis=0) # get mean of preds for each data in the passes
        # evaluate against labels
        y = y_test[(batch_id*batch_size):((batch_id + 1)*batch_size)]
        # compute error
        err += np.count_nonzero(np.not_equal(mean_y.argmax(axis=1), y.argmax(axis=1))) # count the number of wrong classifications (label is 1 and i said 2 after mc)

    err = err / X_test.shape[0]
    net.set_mc_dropout_rate(0)

    return 1. - err


def mc_dropout(net, X_test, batch_size=1000, dropout=0.5, T=100):
    """
    net: keras model with set_mc_dropout_rate function
    
    Forward passes T times, then take the variance from all the predictions for each class.
    the mc_dropout score for an example will be the mean of the variances for all the classes.  
    """
    net.set_mc_dropout_rate(dropout)
    model = net.model
    repititions = []
    for i in tqdm(range(T)):
        pred = model.predict(X_test, batch_size)
        repititions.append(pred)
    net.set_mc_dropout_rate(0)

    repititions = np.array(repititions) # T x btach x pred
    mc = np.var(repititions, axis=0) # get variance from all preds for each example (output: batch x preds classes) each cell is var
    mc = np.mean(mc, axis=-1) # mean of vars of each class (out: one dim array with batch as dim)
    return -mc

# fit & predict

In [7]:
# load the data
(X_train, y_train), (X_test, y_test) = prepare_data()

# prepare the model
lenet = LeNet(
    input_shape=X_train.shape[1:],
    num_classes=10,
)

# train
lenet.set_mc_dropout_rate(0.5)
lenet.train(X_train, y_train, X_test, y_test, epochs=2, verbose=2)

Training with mc_dropout_rate = 0.5.

Train on 60000 samples, validate on 10000 samples
Epoch 1/2
 - 16s - loss: 0.1535 - acc: 0.9530 - val_loss: 0.0744 - val_acc: 0.9761
Epoch 2/2
 - 16s - loss: 0.0675 - acc: 0.9802 - val_loss: 0.0542 - val_acc: 0.9824


In [8]:
acc = evalute_mc(lenet, X_test, y_test, mc_dropout_rate=0.5)
print("acc = ", acc)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:19<00:00,  1.94s/it]


acc =  0.9881


In [9]:
wtf = np.random.rand(*X_test[:1].shape)

In [10]:
ok_socre = mc_dropout(lenet, X_test[:1])
wtf_score = mc_dropout(lenet, wtf)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 1566.72it/s]


In [11]:
print(ok_socre)
print(wtf_score)

[-6.2906408e-12]
[-0.02490894]
